In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn

c:\Users\17539\anaconda3\envs\nlpasm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


先将大模型load进来

并且取出对应的词嵌入

```
Qwen2Config {
  "_name_or_path": "D:/models/Qwen2.5-7B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 152064
}
```

In [3]:
model_name = "D:/models/Qwen2.5-7B-Instruct"

# 创建模型实例
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# 取出对应的模型词嵌入
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 15.05it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [4]:
from transformers import Qwen2Model
model = Qwen2Model.from_pretrained(
    model_name,
    torch_dtype = 'auto',
    device_map = 'auto'
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  4.48it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [6]:
for name, param in model.named_parameters():
    # split out the layer number
    name_l = name.split('.')
    layer_num = name_l[2] if name_l[1] == 'layers' else None
    
    if layer_num is not None:
        # frozen the lower layers to preserve the features
        # train the higher layers to fit into the specific downstream task
        layer_num = int(layer_num)
        if layer_num < 23:
            param.requires_grad = False
        elif layer_num >= 23:
            param.requires_grad = True


In [7]:
import glob
dataset_dir = 'D:/info/program/NLP/asm/dataset/'

train_list = glob.glob(dataset_dir + 'train*.json')
val_list = glob.glob(dataset_dir + 'val*.json')
len(train_list), len(val_list)

(10, 10)

In [8]:
batch_size = 4
epochs = 10
lr = 5e-5
sequence_length = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from ruozhi_dataset import Ruozhi_dataset
from torch.utils.data import DataLoader

train1 = Ruozhi_dataset(train_list[0], tokenizer = tokenizer, sequence_length = sequence_length)
val1 = Ruozhi_dataset(val_list[0], tokenizer = tokenizer, sequence_length = sequence_length)

train_loader1 = DataLoader(train1, batch_size=batch_size, shuffle=True)
val_loader1 = DataLoader(val1, batch_size=len(val1), shuffle=True)

In [10]:
import torch.optim as optim
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

In [1]:
import datetime
print(datetime.datetime.now())

2024-11-01 14:54:57.661644


In [1]:
model.to(device)
print("Start training")
for epoch in range(1):
    print("{} Epoch {} start".format(datetime.datetime.now(), epoch))
    i = 0
    for input_token, input_masking, output_token in train_loader1:
        i += 1
        input_token = input_token.squeeze(1)
        input_masking = input_masking.squeeze(1)
        output_token = output_token.squeeze(1)

        input_token = input_token.to(device)
        input_masking = input_masking.to(device)
        output_token = output_token.to(device)
        
        
        output = model(input_ids = input_token, 
                       attention_mask = input_masking,
                       labels = output_token)
        loss = output.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f"{datetime.datetime.now()} Epoch {epoch + 1}, Training Loss: {loss.item()}")
    # for input_token, output_token in val_loader1:
    #     with torch.no_grad():
    #         input_token = input_token.view(batch_size, sequence_length)
    #         output_token = output_token.view(batch_size, sequence_length)

    #         input_token = input_token.to(device)
    #         output_token = output_token.to(device)


    #         output = model(input_ids = input_token, labels = output_token)
    #         loss = output.loss

    #         print(f"{datetime.datetime.now()} Epoch {epoch + 1}, Validation Loss: {loss.item()}")

Start training


NameError: name 'train_loader1' is not defined

In [13]:
for name, param in model.named_parameters():
    print(f"{name}{param.size()}")

model.embed_tokens.weighttorch.Size([152064, 3584])
model.layers.0.self_attn.q_proj.weighttorch.Size([3584, 3584])
model.layers.0.self_attn.q_proj.biastorch.Size([3584])
model.layers.0.self_attn.k_proj.weighttorch.Size([512, 3584])
model.layers.0.self_attn.k_proj.biastorch.Size([512])
model.layers.0.self_attn.v_proj.weighttorch.Size([512, 3584])
model.layers.0.self_attn.v_proj.biastorch.Size([512])
model.layers.0.self_attn.o_proj.weighttorch.Size([3584, 3584])
model.layers.0.mlp.gate_proj.weighttorch.Size([18944, 3584])
model.layers.0.mlp.up_proj.weighttorch.Size([18944, 3584])
model.layers.0.mlp.down_proj.weighttorch.Size([3584, 18944])
model.layers.0.input_layernorm.weighttorch.Size([3584])
model.layers.0.post_attention_layernorm.weighttorch.Size([3584])
model.layers.1.self_attn.q_proj.weighttorch.Size([3584, 3584])
model.layers.1.self_attn.q_proj.biastorch.Size([3584])
model.layers.1.self_attn.k_proj.weighttorch.Size([512, 3584])
model.layers.1.self_attn.k_proj.biastorch.Size([512])